# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


### Mark all launch sites on a map


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, site in launch_sites_df.iterrows():
    coords = [site['Lat'], site['Long']]
    circle = folium.Circle(coords, radius = 1000, color = '#b50497', fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.map.Marker(coords, icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), html='<div style="font-size: 15; color:#b50497;"><b>%s</b></div>' % site['Launch Site'], ))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

***The launch sites are not in proximity to the Equator line, but are all very close in proximity to their respective coastlines.***

### Mark the success/failed launches for each site on the map


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


##### Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [26]:
marker_cluster = MarkerCluster()

def get_marker_color(class_value):
    if class_value == 1:
        return 'green'
    elif class_value == 0:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)
spacex_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


##### For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [13]:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker=folium.Marker(location=[record['Lat'], record['Long']], popup=record['marker_color']).add_to(marker_cluster)
    icon=folium.Icon(color='white',icon_color=record['marker_color']).add_to(marker)
    marker_cluster.add_child(marker)

site_map

### Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [17]:
# find coordinate of the closet coastline
launch_sites_df.iloc[3]

Launch Site    VAFB SLC-4E
Lat              34.632834
Long           -120.610745
Name: 3, dtype: object

In [18]:
vafb_ls_lat = 34.632834
vafb_ls_lon = -120.610745
coast_lat = 34.63644
coast_lon = -120.62486
distance_coastline = calculate_distance(vafb_ls_lat, vafb_ls_lon, coast_lat, coast_lon)
print("The distance between Launch Site VAFB SLC-4E and the nearest coastline is: ", distance_coastline, 'km')

The distance between Launch Site VAFB SLC-4E and the nearest coastline is:  1.3526287816336575 km


In [19]:
coordinate = [coast_lat, coast_lon]

distance_marker = folium.Marker(
    coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

##### Draw a `PolyLine` between a launch site to the selected coastline point


In [20]:
lines=folium.PolyLine(locations= [[vafb_ls_lat,vafb_ls_lon],coordinate], weight=1)
site_map.add_child(lines)
site_map

##### Draw a line betwee a launch site to its closest city, railway, highway, etc.

In [21]:
lompoc_lat = 34.63898
lompoc_long = -120.45793
railway_lat = 34.63602
railway_long = -120.62392
highway_lat = 34.6391
highway_long = -120.47551

distance_city = calculate_distance(vafb_ls_lat, vafb_ls_lon, lompoc_lat, lompoc_long)
print("The distance from Launch Site VAFB SLC-4E to the nearest city, Lompoc, is ", distance_city, "km")

distance_rail = calculate_distance(vafb_ls_lat, vafb_ls_lon, railway_lat, railway_long)
print("The distance from Launch Site VAFB SLC-4E to the nearest railway is ", distance_rail, "km")

distance_highway = calculate_distance(vafb_ls_lat, vafb_ls_lon, highway_lat, highway_long)
print("The distance from Launch Site VAFB SLC-4E to the nearest highway, Ocean Ave, is ", distance_highway, "km")

The distance from Launch Site VAFB SLC-4E to the nearest city, Lompoc, is  14.001977170252344 km
The distance from Launch Site VAFB SLC-4E to the nearest railway is  1.2567653524754498 km
The distance from Launch Site VAFB SLC-4E to the nearest highway, Ocean Ave, is  12.395997999050568 km


In [22]:
coordinate1 = [lompoc_lat, lompoc_long]
distance_marker = folium.Marker(
    coordinate1,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
    )
site_map.add_child(distance_marker)


coordinate2 = [railway_lat, railway_long]
distance_marker = folium.Marker(
    coordinate2,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rail),
        )
    )
site_map.add_child(distance_marker)


coordinate3 = [highway_lat, highway_long]
distance_marker = folium.Marker(
    coordinate3,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
    )
site_map.add_child(distance_marker)

In [23]:
lines=folium.PolyLine(locations= [[vafb_ls_lat,vafb_ls_lon],coordinate1], weight=1)
site_map.add_child(lines)

lines=folium.PolyLine(locations= [[vafb_ls_lat,vafb_ls_lon],coordinate2], weight=1)
site_map.add_child(lines)

lines=folium.PolyLine(locations= [[vafb_ls_lat,vafb_ls_lon],coordinate3], weight=1)
site_map.add_child(lines)

site_map

***With regard to the VAFB SLC-4E launch site, it is pretty close to the railway and coastline, almost equally distant from both at 1.26km and 1.35km respectively. The nearest highway would be the CA 246 highway as you enter Lompoc, which happens to be the nearest city as well. However, they do have a good amount of distance, being upwards of 12km away from the luanch site.***